In [1]:
import pandas as pd

# Load the Excel file
file_path = "C:/Users/Alok Kumar Pandey/Downloads/Book1.xlsx"
df = pd.read_excel(file_path)

# Check for NaN values
nan_check = df.isnull().values.any()

if nan_check:
    print("There are NaN values in the Excel sheet.")
else:
    print("There are no NaN values in the Excel sheet.")

There are no NaN values in the Excel sheet.


In [3]:
from sklearn.model_selection import train_test_split

# Define your features and target variable (assuming 'target' is your target column)
X = df.drop(columns=['Leq'])  # Replace 'target' with your actual target column name
y = df['Leq']  # Replace 'target' with your actual target column name


In [4]:

# Split the data into train (80%) and test (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the sizes of the train and test sets
print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

Training set size: 368
Test set size: 92


In [6]:
# # Convert columns with string numbers to proper numeric values
# df = df.replace(',', '.', regex=True)  # Replace commas with dots for decimal separation
# df = df.apply(pd.to_numeric, errors='coerce')  # Convert all columns to numeric, setting errors to NaN for non-numeric values

# # Calculate the correlation matrix
# correlation_matrix = df.corr()

# # Display the correlation matrix
# print("Correlation matrix:\n", correlation_matrix)

# # Set a threshold for high correlation (e.g., 0.8)
# threshold = 0.8

# # Find pairs of columns with correlation greater than the threshold
# high_corr_pairs = []

# for i in range(len(correlation_matrix.columns)):
#     for j in range(i):
#         if abs(correlation_matrix.iloc[i, j]) > threshold:  # Check absolute correlation
#             colname1 = correlation_matrix.columns[i]
#             colname2 = correlation_matrix.columns[j]
#             high_corr_pairs.append((colname1, colname2, correlation_matrix.iloc[i, j]))

# # Print highly correlated pairs
# if high_corr_pairs:
#     print("\nHighly correlated columns (correlation > 0.8):")
#     for col1, col2, corr_value in high_corr_pairs:
#         print(f"{col1} and {col2}: {corr_value:.2f}")
# else:
#     print("No highly correlated columns found with correlation greater than 0.8.")


Correlation matrix:
                      Leq       L10       L50       L90    Speed        Car  \
Leq             1.000000  0.867535  0.883498  0.858651 -0.145071  0.600849   
L10             0.867535  1.000000  0.963669  0.956383 -0.126770  0.528490   
L50             0.883498  0.963669  1.000000  0.984782 -0.133415  0.645415   
L90             0.858651  0.956383  0.984782  1.000000 -0.117051  0.587749   
Speed          -0.145071 -0.126770 -0.133415 -0.117051  1.000000 -0.050590   
Car             0.600849  0.528490  0.645415  0.587749 -0.050590  1.000000   
2w              0.487152  0.378598  0.508919  0.433392 -0.238409  0.756933   
3w             -0.220730 -0.353472 -0.274342 -0.335998  0.051276  0.132580   
LCV             0.593836  0.603268  0.591789  0.565142 -0.058767  0.467244   
Heavy traffic   0.691045  0.781455  0.730931  0.738651  0.002463  0.356019   
%HT             0.095292  0.238431  0.103548  0.170841  0.148986 -0.417880   
Median width    0.386782  0.257237  0.41001

In [10]:
import pandas as pd
import numpy as np  # Import NumPy

# Load the Excel file
file_path = "C:/Users/Alok Kumar Pandey/Downloads/Book1.xlsx"
df = pd.read_excel(file_path)

# Convert columns with string numbers to proper numeric values
df = df.replace(',', '.', regex=True)  # Replace commas with dots for decimal separation
df = df.apply(pd.to_numeric, errors='coerce')  # Convert all columns to numeric, setting errors to NaN for non-numeric values

# Drop columns that are completely non-numeric or filled with NaN after conversion
df_cleaned = df.dropna(axis=1, how='all')

# Calculate the correlation matrix
correlation_matrix = df_cleaned.corr()

# Set a correlation threshold
threshold = 0.8

# Create an upper triangle correlation matrix to avoid duplicate pairs
upper_triangle = correlation_matrix.where(
    np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))  # Using NumPy here

# Find columns with a correlation above the threshold
columns_to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column].abs() > threshold)]

# Drop highly correlated columns
df_reduced = df_cleaned.drop(columns=columns_to_drop)

# Print the columns to drop and the resulting DataFrame
print("Columns to drop due to high correlation:\n", columns_to_drop)
print("\nDataFrame after dropping highly correlated columns:\n", df_reduced)


Columns to drop due to high correlation:
 ['L10', 'L50', 'L90', 'Median width', 'Shoulder width', 'Temp.']

DataFrame after dropping highly correlated columns:
            Leq     Speed   Car   2w  3w  LCV  Heavy traffic       %HT  RH  WS
0    72.582436  61.808820  210  237   6   10             17  3.541667  50  11
1    72.725820  59.440556  207  219   6   15             20  4.282655  51  12
2    72.576617  60.354065  200  236   2   11             19  4.059829  52  13
3    72.097628  63.814342  237  262   5   13             20  3.724395  53  14
4    72.446424  67.122706  252  260  10    7             18  3.290676  54  15
..         ...        ...  ...  ...  ..  ...            ...       ...  ..  ..
455  79.191706  67.733449  532  318   7   58             68  6.917599  40  20
456  78.006703  69.533363  446  344   4   31             85  9.340659  37  15
457  79.325576  69.191092  548  278   1   35             79  8.395324  36  15
458  79.978307  67.314046  522  393   3   44             64

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.inspection import permutation_importance

# Load the data (replace with your file path)
file_path = 'reduced_data_traffic.xlsx'
data = pd.read_excel(file_path)

# Defining the features (all columns except Leq) and target (Leq)
X = data.drop(columns=['Leq'])
y = data['Leq']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build a simple ANN model
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[X_train_scaled.shape[1]]),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Output layer for regression
])

# Compile the model with an optimizer and loss function
model.compile(optimizer='adam', loss='mse')

# Create a list to store the training log
training_logs = []

# Custom callback to log the training output
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        training_logs.append(logs)

# Train the model with the custom callback
history = model.fit(X_train_scaled, y_train, epochs=100, validation_split=0.2, verbose=0, callbacks=[CustomCallback()])

# Display the complete training logs
for epoch, log in enumerate(training_logs):
    print(f"Epoch {epoch + 1}: {log}")

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Calculate the MSE, MAE, and RMSE
mse_ann = mean_squared_error(y_test, y_pred)
mae_ann = mean_absolute_error(y_test, y_pred)
rmse_ann = np.sqrt(mse_ann)

print(f"ANN Results:\nMSE: {mse_ann}\nMAE: {mae_ann}\nRMSE: {rmse_ann}")

# Create a wrapper class for the Keras model to use with sklearn's permutation_importance
class KerasRegressorWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y, epochs=100, verbose=0):
        self.model.fit(X, y, epochs=epochs, verbose=verbose)
        return self

    def predict(self, X):
        return self.model.predict(X).flatten()

    def score(self, X, y):
        return -mean_squared_error(y, self.predict(X))  # Returning negative MSE as score

# Wrap the Keras model
wrapped_model = KerasRegressorWrapper(model)

# Fit the wrapped model
wrapped_model.fit(X_train_scaled, y_train)

# Apply permutation importance on the trained ANN model
perm_importance = permutation_importance(wrapped_model, X_test_scaled, y_test, n_repeats=10, random_state=42)

# Displaying feature importances
feature_importances = pd.Series(perm_importance.importances_mean, index=X.columns).sort_values(ascending=False)
print("Feature Importances (Permutation Importance):")
print(feature_importances)


C:\Users\Alok Kumar Pandey\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1: {'loss': 5698.52685546875, 'val_loss': 5642.6240234375}
Epoch 2: {'loss': 5598.47265625, 'val_loss': 5542.107421875}
Epoch 3: {'loss': 5489.91162109375, 'val_loss': 5424.58349609375}
Epoch 4: {'loss': 5356.46826171875, 'val_loss': 5272.0361328125}
Epoch 5: {'loss': 5180.822265625, 'val_loss': 5068.275390625}
Epoch 6: {'loss': 4950.66455078125, 'val_loss': 4805.423828125}
Epoch 7: {'loss': 4656.384765625, 'val_loss': 4469.248046875}
Epoch 8: {'loss': 4285.85888671875, 'val_loss': 4057.744140625}
Epoch 9: {'loss': 3842.3095703125, 'val_loss': 3570.68994140625}
Epoch 10: {'loss': 3329.353759765625, 'val_loss': 3019.860107421875}
Epoch 11: {'loss': 2760.737548828125, 'val_loss': 2428.920654296875}
Epoch 12: {'loss': 2170.19580078125, 'val_loss': 1836.3973388671875}
Epoch 13: {'loss': 1599.7210693359375, 'val_loss': 1290.84130859375}
Epoch 14: {'loss': 1097.771728515625, 'val_loss': 846.4442138671875}
Epoch 15: {'loss': 716.6375122070312, 'val_loss': 532.8558959960938}
Epoch 16: {'